#### Imports

In [ ]:
import json
import os

def add_iscrowd(path):

    with open(path, 'r') as f:
        data = json.load(f)

    for i, annot in enumerate(data['annotations']):
        annot['iscrowd'] = 0
        data['annotations'][i] = annot

    with open(path, 'w') as outfile:
        json.dump(data, outfile)

base = ...

for ds in ['duke_train', 'duke_val', 'manual_maxar_val', 'fake_maxar_train', 'fake_maxar_val']:
    path = os.path.join(base, ds, 'labels.json') 
    add_iscrowd(path)

In [ ]:
# torch
import torch
import torchvision

# Detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Common Libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

# Detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

#### Register and check datasets

In [ ]:
os.chdir(os.path.join(os.getcwd(), '..', 'datasets'))

In [ ]:
from itertools import product

for d, ds in product(["train", "val"], ['duke']):
    ds_path = os.path.join(os.getcwd(), f'{ds}_{d}', 'data')
    json_path = os.path.join(os.getcwd(), f'{ds}_{d}', 'labels.json')
    ds_name = f'{ds}_{d}'

    if ds == 'overfit':
        continue

    if ds_name in DatasetCatalog.list():
        DatasetCatalog.remove(ds_name)
        MetadataCatalog.remove(ds_name)

    register_coco_instances(ds_name, {}, json_path, ds_path)


ds_name = 'iscrowd_manual_maxar_val'   
ds_path = os.path.join(os.getcwd(), f'{ds_name}', 'data')
json_path = os.path.join(os.getcwd(), f'{ds_name}', 'labels.json')

if ds_name in DatasetCatalog.list():
    DatasetCatalog.remove(ds_name)
    MetadataCatalog.remove(ds_name)

register_coco_instances(ds_name, {}, json_path, ds_path)

In [ ]:
train_dataset = 'duke_train'
val_dataset = 'duke_val'
train_metadata = MetadataCatalog.get(train_dataset)
val_metadata = MetadataCatalog.get(val_dataset)

In [ ]:
%matplotlib inline
dataset_dicts = DatasetCatalog.get(train_dataset)
tower_metadata = MetadataCatalog.get(train_dataset)

num_examples = 5
for d in random.sample(dataset_dicts, num_examples):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=tower_metadata, scale=0.8, instance_mode=1)
    out = visualizer.draw_dataset_dict(d)
    # cv2.imshow(out.get_image()[:, :, ::-1])
    plt.imshow(out.get_image()[:, :, ::])

#### Setup training config

In [ ]:
!nvidia-smi

In [ ]:
from detectron2.engine import DefaultTrainer


from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.evaluation import inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data import DatasetMapper


class EvalTrainer(DefaultTrainer):
    def __init__(self, cfg):
        super().__init__(cfg)

        self.same_data_loader = build_detection_test_loader(DatasetCatalog.get(cfg.DATASETS.TEST1),
                                                            mapper=DatasetMapper(cfg, is_train=False))
        self.same_data_eval = COCOEvaluator(cfg.DATASETS.TEST1)
        self.manual_maxar_loader = build_detection_test_loader(DatasetCatalog.get(cfg.DATASETS.TEST2),
                                                            mapper=DatasetMapper(cfg, is_train=False))
        self.manual_maxar_eval = COCOEvaluator(cfg.DATASETS.TEST2)


    def after_step(self):
        super().after_step()

        if self.iter % self.cfg.TEST.INTERVAL == 0:
            
            same_data_results = inference_on_dataset(self.model, 
                                                     self.same_data_loader,
                                                     self.same_data_eval)


            with open('results_samedata_'+str(self.cfg.DATASETS.TRAIN)+str(self.iter)+'.json', 'w') as out:
                json.dump(same_data_results, out)                                   

            results = inference_on_dataset(self.model, 
                                                     self.manual_maxar_loader,
                                                     self.manual_maxar_eval)

            with open('results_manualsdata_'+str(self.cfg.DATASETS.TRAIN)+str(self.iter)+'.json', 'w') as out:
                json.dump(results, out)                                   

            

    



from datetime import date

def do_train(train_dataset='duke'):

    cfg = get_cfg() # Model Confi


    frcnn= 'faster_rcnn_R_101_FPN_3x.yaml'

    # From Detectron2 Model Zoo
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/"+frcnn))    # https://github.com/facebookresearch/detectron2/blob/main/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/"+frcnn)  # Pre-trained Model Weights

    cfg.DATALOADER.NUM_WORKERS = 2  # Number of CPUs to load the data into Detectron2 - 2 for Colab
    cfg.SOLVER.IMS_PER_BATCH = 2    # Detectron2 default 16 with 8 GPUs, so 16/8 = 2 for 1 GPU

    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2   # for R-CNN Models

    cfg.defrost()

    cfg.DATASETS.TRAIN = (train_dataset+'_train')
    cfg.DATASETS.TEST1 = (train_dataset+'_val')
    cfg.DATASETS.TEST2 = ('manual_maxar_val')
    cfg.TEST.INTERVAL = 30
    model_name = str(date.today()) + '_dummy'
    cfg.OUTPUT_DIR = '/content/drive/MyDrive/PyPSA_Africa_images/models/' + model_name

    cfg.freeze()

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = EvalTrainer(cfg) 
    trainer.resume_or_load(resume=False)

    trainer.train()



In [ ]:
trainer.train()